Creates a lazy xarray from a STAC collection and saves the result to a COG file

In [ ]:
#!pip install boto3 rasterio stackstac pystac pystac_client numpy rioxarray

In [ ]:
from boto3.session import Session
from rasterio.session import AWSSession
import stackstac
import pystac
import os
import rioxarray

In [ ]:
aws_access_key_id=os.environ.get('aws_access_key_id')
aws_secret_access_key=os.environ.get('aws_secret_access_key')
endpoint_url=os.environ.get('endpoint_url','s3.us-east.cloud-object-storage.appdomain.cloud')
stac_endpoint=os.environ.get('stac_endpoint', "https://stac-fastapi-pgstac-nasageospatial-dev.cash.sl.cloud9.ibm.com")
collection_id=os.environ.get('collection_id','sentinel-1-grd')

# cog file target location (path/)filename.cog
save_to=os.environ.get('save_to',"cogfile.cog")

# limit number of returned stac items
limit=int(os.environ.get('limit', '2'))

In [ ]:
from pystac_client import Client

client = Client.open(stac_endpoint)

s1_coll = client.get_collection(collection_id)

items = []

In [ ]:
item = next(s1_coll.get_items(), None)

i = 0

while item is not None:
    items.append(item.to_dict())
    i = i + 1
    if i >= limit:
        break
    item = next(s1_coll.get_items(), None)

In [ ]:
session = Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
)

In [ ]:
aws_session = AWSSession(
    session=session,
    endpoint_url=endpoint_url,
)

In [ ]:
data_array = stackstac.stack(items, epsg=4326, resolution=20, gdal_env=stackstac.DEFAULT_GDAL_ENV.updated(always=dict(session=aws_session)))

In [ ]:
data_array

In [ ]:
#data_array.isel(time=0).sel(band='data').plot()

In [ ]:
a = data_array.compute()

In [ ]:
a

In [ ]:
a.isel(time=0).rio.to_raster(raster_path=save_to, driver="COG")